In [1]:
!pip install torch==2.2.2
!pip install torch-cluster==1.6.3
!pip install torch-geometric==2.6.1
!pip install torch-scatter==2.1.2
!pip install torch-sparse==0.6.18
!pip install torch-spline-conv==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.3 MB/s eta 0:00:00:00:0100:01
   ━━

In [2]:
!pip install ogb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.3 MB/s eta 0:00:00


In [3]:
# Importing necessary dependencies in order to import our dataset, create our
# GCN models, and evaluate the models

import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU

from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GINConv, GATConv

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from torch_geometric.data.data import DataEdgeAttr, GlobalStorage

from torch_geometric.nn import Node2Vec

import pandas as pd
import shutil, os
import os.path as osp
import numpy as np

#from logger import Logger
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import random

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

Device: cuda


In [5]:
# loaded with transform parameter set as such in order to obtain the adj_t matrix
# required for the GNN layers
# torch.serialization.add_safe_globals([DataEdgeAttr])
# torch.serialization.add_safe_globals([GlobalStorage])
dataset = PygLinkPropPredDataset(name='ogbl-ddi', root ='./', transform=T.ToSparseTensor()) # loading ogb-ddi
print('Task type: {}'.format(dataset.task_type))
graph = dataset[0]
adj_t = graph.adj_t.to(device) # loads all edges in graph into sparse adj_t matrix

Downloaded 0.04 GB: 100%|██████████| 46/46 [00:02<00:00, 22.36it/s]


Extracting ./ddi.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 64.77it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 3587.94it/s]

Saving...
Task type: link prediction



Done!


In [6]:
# getting the train, validation, and test edge splits
split_edge = dataset.get_edge_split()
train_edges = split_edge['train']['edge']
torch.manual_seed(70) # picking random samples to evaluate on
idx = torch.randperm(split_edge['train']['edge'].size(0))
idx = idx[:split_edge['valid']['edge'].size(0)]
split_edge['eval_train'] = {'edge': split_edge['train']['edge'][idx]}

In [7]:
train_edges_node2vec = train_edges.T # transpose to get the right dimension
# Initialize edge-induced subgraph with only train edges (edge-induced subgraph)
data_node2vec = Data(edge_index=train_edges_node2vec)

In [8]:
def save_embedding(model, filepath): # function to save embedding to specified filepath
    torch.save(model.embedding.weight.data.cpu(), filepath)

In [9]:
import os
if not os.path.exists('/kaggle/working/training_outputs/'):
    os.makedirs('/kaggle/working/training_outputs/') # directory for saving visualizations and model checkpoints

In [10]:
# class in order to predict whether a link exists between two nodes using
# their embeddings, x_i and x_j
class LinkPredictor(torch.nn.Module):
    ''' Neural network which predicts whether a link (interaction) exists between 2 nodes i,j
    given their embeddings x_i, x_j.
    '''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j # hadamard product
        for lin in self.lins[:-1]: # linear layer -> relu -> dropout
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x) # sigmoid activation outputs probability that a given edge exists for all node pairs

In [11]:
def train(model, predictor, x, adj_t, split_edge, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = split_edge['train']['edge'].to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        # computes the loss for positive edges
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        # samples negative edges from the graph
        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')

        # computes the loss for negative edges
        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, x, adj_t, split_edge, evaluator, batch_size):
    model.eval()
    predictor.eval()

    h = model(x, adj_t)

    pos_train_edge = split_edge['eval_train']['edge'].to(x.device)
    pos_valid_edge = split_edge['valid']['edge'].to(x.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(x.device)
    pos_test_edge = split_edge['test']['edge'].to(x.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(x.device)

    # store what the link predictor outputs for each positive and negative
    # edge in order to compute the hits@K
    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    # compute the hits@K for training, validation, and test
    results = {}
    for K in [10, 20]:
        evaluator.K = K
        train_hits = evaluator.eval({
            'y_pred_pos': pos_train_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (train_hits, valid_hits, test_hits)

    return results


In [27]:
gnn_args = { # define GNN hyperparams
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'hidden_size': 256,
    'dropout': 0.5,
    'epochs': 100,
    'weight_decay': 1e-5,
    'lr': 0.005,
    'attn_size': 32,
    'num_layers':2,
    'log_steps':1,
    'eval_steps':5,
    'runs':10,
    'batch_size': 16*1024,
    'attn_heads': 1,

}

In [28]:
def train_model(model, emb, gnn_args, predictor, model_name):
  '''
  Train specified GNN model. Model and embeddings should be initialized.
  Save model after every run.
  '''
  train_hits_arr, val_hits_arr, test_hits_arr = [], [], []

  evaluator = Evaluator(name='ogbl-ddi')
  for run in range(2):
    max_valhits, train_hits_run, test_hits_run = float('-inf'), 0, 0

    torch.nn.init.xavier_uniform_(emb.weight)
    model.reset_parameters()
    predictor.reset_parameters()
    optimizer = torch.optim.Adam(
        list(model.parameters()) + list(emb.parameters()) +
        list(predictor.parameters()), lr=gnn_args['lr'])

    for epoch in range(1, 1 + gnn_args['epochs']):
        loss = train(model, predictor, emb.weight, adj_t, split_edge,
                      optimizer, gnn_args['batch_size'])

        if epoch % gnn_args['eval_steps'] == 0:
            results = test(model, predictor, emb.weight, adj_t, split_edge,
                            evaluator, gnn_args['batch_size'])


            if epoch % gnn_args['log_steps'] == 0:
                for key, result in results.items():
                    train_hits, valid_hits, test_hits = result
                    print(key)
                    print(f'Run: {run + 1:02d}, '
                          f'Epoch: {epoch:02d}, '
                          f'Loss: {loss:.4f}, '
                          f'Train: {100 * train_hits:.2f}%, '
                          f'Valid: {100 * valid_hits:.2f}%, '
                          f'Test: {100 * test_hits:.2f}%')
                print('---')

            # check val-hits@20
            train_hits, valid_hits, test_hits = results['Hits@20']
            if valid_hits >= max_valhits: # if validhits20 is higher than max, save ckpt
              max_valhits = valid_hits
              train_hits_run = train_hits
              test_hits_run = test_hits
              # Save model checkpoint for current run.
              model_path = f"training_outputs/{model_name}.pt"
              emb_path = f'training_outputs/{model_name}_init_emb.pt'
              save_model_ckpt(model, emb, optimizer, predictor, loss, emb_path, model_path)
    train_hits_arr.append(train_hits_run)
    test_hits_arr.append(test_hits_run)
    val_hits_arr.append(max_valhits)


  # Print overall stats arrays for best model based on val hits@20
  print("Val_hits@20: ", val_hits_arr)
  print("Test_hits@20: ", test_hits_arr)
  print("Train_hits@20: ", train_hits_arr)

  # Print best model stats (based on val hits@20)
  val_max = max(val_hits_arr)
  print("Best model val hits@20: ", max(val_hits_arr))
  max_idx = val_hits_arr.index(val_max)
  print('Best model test hits@20: ', test_hits_arr[max_idx])
  print('Best model train hits@20: ', val_hits_arr[max_idx])

  # convert to numpy array
  val_hits_arr = np.array(val_hits_arr)
  test_hits_arr = np.array(test_hits_arr)
  train_hits_arr = np.array(train_hits_arr)

  # Print average stats + variance
  print(f"Average best train hits@20: {np.mean(train_hits_arr)}; var: {np.var(train_hits_arr)}")
  print(f"Average best val hits@20: {np.mean(val_hits_arr)}; var: {np.var(val_hits_arr)}")
  print(f"Average best test hits@20: {np.mean(test_hits_arr)}; var: {np.var(test_hits_arr)}")

In [29]:
def save_model_ckpt(model, emb, optimizer, predictor, loss, emb_path, model_path):
  ''' Save model and embedding checkpoints. '''
  EPOCH = 100
  # Save model params
  torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'predictor_state_dict': predictor.state_dict(),
            'loss': loss,
            }, model_path)
  # Also save initial embedding (just in case)
  torch.save(emb.weight.data.cpu(), emb_path)

In [30]:
def load_model_ckpt(curr_model, model_name, run):
  ''' Load model checkpoint. '''
  evaluator = Evaluator(name='ogbl-ddi')
  model_path = f"training_outputs/{model_name}.pt"
  emb_path = f'training_outputs/{model_name}_init_emb.pt'

  # Load emb (init feature representations)
  pretrained_weight = torch.load(emb_path, map_location='cpu').to(device)
  pretrained_weight = pretrained_weight.cpu().data.numpy()
  emb_after = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
  # Pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
  emb_after.weight.data.copy_(torch.from_numpy(pretrained_weight))

  # Init optimizer and predictor objects
  predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
  optimizer = torch.optim.Adam(
        list(curr_model.parameters()) + list(emb_after.parameters()) +
        list(predictor.parameters()), lr=gnn_args['lr'])


  # Load model, predictor, and optimizer params
  checkpoint = torch.load(model_path)
  curr_model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  predictor.load_state_dict(checkpoint['predictor_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']


  # Save final embedding representation of all nodes
  h = curr_model(emb_after.weight, adj_t)
  final_emb_path = f'training_outputs/{model_name}_final_emb_{run}.pt'
  torch.save(h, final_emb_path)

  # Evaluate pretrained model
  results = test(curr_model, predictor, emb_after.weight, adj_t, split_edge,
                               evaluator, gnn_args['batch_size'])

  # Print hits stats
  for key, result in results.items():
    print(key)
    train_hits, valid_hits, test_hits = result
    print(f'Train: {100 * train_hits:.2f}%, '
                              f'Valid: {100 * valid_hits:.2f}%, '
                              f'Test: {100 * test_hits:.2f}%')

In [31]:
class GAT(torch.nn.Module):
    '''Define GAT network using PyG's GATConv layer.'''
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                dropout=0.5, heads=1):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout))
        
        for _ in range(num_layers - 2):
            self.convs.append(
                GATConv(hidden_channels * heads, hidden_channels, heads=heads, dropout=dropout)
            )
        
        self.convs.append(GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index)
        return x

In [32]:
# TRAIN gcn with random features
model_name = 'gat_rand_feat'
gat_model = GAT(in_channels = gnn_args['hidden_size'],hidden_channels = 32,out_channels = gnn_args['hidden_size'],
              num_layers = gnn_args['num_layers'], dropout = gnn_args['dropout'], heads = 1).to(device)
gat_predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gat_emb_rand = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
train_model(gat_model, gat_emb_rand, gnn_args, gat_predictor, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.5427, Train: 16.26%, Valid: 15.43%, Test: 5.20%
Hits@20
Run: 01, Epoch: 05, Loss: 0.5427, Train: 20.54%, Valid: 19.76%, Test: 9.42%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.4247, Train: 16.63%, Valid: 15.37%, Test: 4.57%
Hits@20
Run: 01, Epoch: 10, Loss: 0.4247, Train: 31.17%, Valid: 29.67%, Test: 14.61%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.3803, Train: 21.70%, Valid: 19.39%, Test: 8.61%
Hits@20
Run: 01, Epoch: 15, Loss: 0.3803, Train: 35.06%, Valid: 32.16%, Test: 12.96%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.3549, Train: 18.46%, Valid: 16.38%, Test: 1.17%
Hits@20
Run: 01, Epoch: 20, Loss: 0.3549, Train: 34.16%, Valid: 30.97%, Test: 7.83%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.3341, Train: 24.10%, Valid: 21.13%, Test: 7.83%
Hits@20
Run: 01, Epoch: 25, Loss: 0.3341, Train: 36.68%, Valid: 32.77%, Test: 15.51%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.3175, Train: 24.35%, Valid: 21.56%, Test: 6.13%
Hits@20
Run: 01, Epoch: 30, Loss: 0.3175, Train: 30.2

In [33]:
# Loading GAT trained on Features
gat_model = GAT(in_channels = gnn_args['hidden_size'],hidden_channels = 32,out_channels = gnn_args['hidden_size'],
              num_layers = gnn_args['num_layers'], dropout = gnn_args['dropout'], heads = 1).to(device)
load_model_ckpt(gat_model, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 39.69%, Valid: 35.02%, Test: 8.49%
Hits@20
Train: 52.24%, Valid: 47.39%, Test: 25.95%


In [34]:
from huggingface_hub import HfApi, HfFolder, upload_file, create_repo, hf_hub_download

In [35]:
from huggingface_hub import login

In [36]:

login()

In [37]:
file_path = hf_hub_download(
    repo_id="Rzoro/ogb_ddi",  # Replace with your model repo
    filename="node2vec_state_dict.pth",          # Replace with your model file
    repo_type="model"                         # Could be 'dataset' or 'space' too
)

print("Downloaded to:", file_path)

node2vec_state_dict.pth:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

Downloaded to: /root/.cache/huggingface/hub/models--Rzoro--ogb_ddi/snapshots/73c5c4f0b4718e53e29226ecf164d2aeddd832a6/node2vec_state_dict.pth


In [38]:
# training function for node2vec using PyG's Node2Vec function
def train_Node2Vec(args, data, filepath):
  model = Node2Vec(data.edge_index, args['embedding_dim'], args['walk_length'],
                    args['context_size'], args['walks_per_node'],
                    sparse=True).to(device)

  loader = model.loader(batch_size=args['batch_size'], shuffle=True,
                        num_workers=4)
  optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=args['lr'])

  model.train()
  for epoch in range(1, args['epochs'] + 1):
      for i, (pos_rw, neg_rw) in enumerate(loader):
          optimizer.zero_grad()
          loss = model.loss(pos_rw.to(device), neg_rw.to(device))
          loss.backward()
          optimizer.step()

          if (i + 1) % args['log_steps'] == 0:
              print(f'Epoch: {epoch:02d}, Step: {i+1:03d}/{len(loader)}, '
                    f'Loss: {loss:.4f}')

          if (i + 1) % 100 == 0:  # Save model every 100 steps.
              save_embedding(model, filepath)
      save_embedding(model, filepath)
  return model

In [39]:
def load_node2vec(args, data, filepath):
    model = Node2Vec(data.edge_index, args['embedding_dim'], args['walk_length'],
                    args['context_size'], args['walks_per_node'],
                    sparse=True).to(device)
    loader = model.loader(batch_size=args['batch_size'], shuffle=True,
                        num_workers=4)
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=args['lr'])

    model.load_state_dict(torch.load(filepath))

    return model

In [40]:
node2vec_args = {'device':0, 'embedding_dim':256, 'walk_length':40, 'context_size':20, 'walks_per_node':10,
      'batch_size':256, 'lr':0.01, 'epochs':100, 'log_steps':1}

In [44]:
new_node2vec2 = load_node2vec(node2vec_args, data_node2vec, file_path)
pretrained_weight = new_node2vec2.embedding.weight.data.cpu()

In [43]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gat_rand_feat.pt",
    path_in_repo="gat_rand_feat.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gat_rand_feat_final_emb_0.pt",
    path_in_repo="gat_rand_feat_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gat_rand_feat_init_emb.pt",
    path_in_repo="gat_rand_feat_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gat_rand_feat.pt:   0%|          | 0.00/9.75M [00:00<?, ?B/s]

gat_rand_feat_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

gat_rand_feat_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/652965602ec912a722be42da2edfbc99888b982c', commit_message='Upload gat_rand_feat_init_emb.pt with huggingface_hub', commit_description='', oid='652965602ec912a722be42da2edfbc99888b982c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)

In [45]:
# Train GAT with node2vec embedding features
model_name = "gat_node2vec_feat_256"
gat_model2 = GAT(in_channels = gnn_args['hidden_size'],hidden_channels = 32,out_channels = gnn_args['hidden_size'],
              num_layers = gnn_args['num_layers'], dropout = gnn_args['dropout'], heads = 1).to(device)

gat_predictor2 = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
gat_emb_node2vec = torch.nn.Embedding(dataset.data.num_nodes, gnn_args['hidden_size']).to(device)
gat_emb_node2vec.weight.data.copy_(pretrained_weight)
train_model(gat_model2, gat_emb_node2vec, gnn_args, gat_predictor2, model_name)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Run: 01, Epoch: 05, Loss: 0.5395, Train: 20.90%, Valid: 19.73%, Test: 7.79%
Hits@20
Run: 01, Epoch: 05, Loss: 0.5395, Train: 27.81%, Valid: 26.50%, Test: 10.66%
---
Hits@10
Run: 01, Epoch: 10, Loss: 0.4246, Train: 25.68%, Valid: 23.78%, Test: 9.45%
Hits@20
Run: 01, Epoch: 10, Loss: 0.4246, Train: 31.74%, Valid: 29.56%, Test: 17.64%
---
Hits@10
Run: 01, Epoch: 15, Loss: 0.3880, Train: 31.81%, Valid: 29.09%, Test: 8.50%
Hits@20
Run: 01, Epoch: 15, Loss: 0.3880, Train: 38.39%, Valid: 35.68%, Test: 17.95%
---
Hits@10
Run: 01, Epoch: 20, Loss: 0.3601, Train: 30.15%, Valid: 27.25%, Test: 8.23%
Hits@20
Run: 01, Epoch: 20, Loss: 0.3601, Train: 37.02%, Valid: 33.92%, Test: 15.36%
---
Hits@10
Run: 01, Epoch: 25, Loss: 0.3376, Train: 27.04%, Valid: 24.03%, Test: 10.53%
Hits@20
Run: 01, Epoch: 25, Loss: 0.3376, Train: 34.88%, Valid: 31.39%, Test: 19.16%
---
Hits@10
Run: 01, Epoch: 30, Loss: 0.3293, Train: 30.23%, Valid: 26.94%, Test: 8.02%
Hits@20
Run: 01, Epoch: 30, Loss: 0.3293, Train: 3

In [46]:
# Loading GAT trained on Features
gat_model2 = GAT(in_channels = gnn_args['hidden_size'],hidden_channels = 32,out_channels = gnn_args['hidden_size'],
              num_layers = gnn_args['num_layers'], dropout = gnn_args['dropout'], heads = 1).to(device)
load_model_ckpt(gat_model2, model_name,0)

/usr/local/lib/python3.11/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Hits@10
Train: 34.05%, Valid: 29.52%, Test: 6.37%
Hits@20
Train: 52.61%, Valid: 46.12%, Test: 14.51%


In [48]:
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gat_node2vec_feat_256.pt",
    path_in_repo="gat_node2vec_feat_256.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gat_node2vec_feat_256_final_emb_0.pt",
    path_in_repo="gat_node2vec_feat_256_final_emb_0.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/kaggle/working/training_outputs/gat_node2vec_feat_256_init_emb.pt",
    path_in_repo="gat_node2vec_feat_256_init_emb.pt",
    repo_id="Rzoro/ogb_ddi",
    repo_type="model"
)

gat_node2vec_feat_256.pt:   0%|          | 0.00/9.75M [00:00<?, ?B/s]

gat_node2vec_feat_256_final_emb_0.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

gat_node2vec_feat_256_init_emb.pt:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rzoro/ogb_ddi/commit/4fb9fbe9622d2562751bad502b91e7498dd8a90f', commit_message='Upload gat_node2vec_feat_256_init_emb.pt with huggingface_hub', commit_description='', oid='4fb9fbe9622d2562751bad502b91e7498dd8a90f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rzoro/ogb_ddi', endpoint='https://huggingface.co', repo_type='model', repo_id='Rzoro/ogb_ddi'), pr_revision=None, pr_num=None)